In [1]:
import folium
import openrouteservice as ors
import math

In [3]:
coords = [
    { 'location': [-87.7898356, 41.8879452]},
    { 'location': [-87.7808524, 41.8906422]},
    { 'location': [-87.7895149, 41.8933762]},
    { 'location': [-87.7552925, 41.8809087]},
    { 'location': [-87.7728134, 41.8804058]},
    { 'location': [-87.7702890, 41.8802231]},
    { 'location': [-87.7787924, 41.8944518]},
    { 'location': [-87.7732345, 41.8770663]},
]
vehicle_start = [-87.800701, 41.876214]
m = folium.Map(location=list(reversed([-87.787984, 41.8871616])), tiles="cartodbpositron", zoom_start=14)
    
folium.Marker(location=list(reversed(vehicle_start)), icon=folium.Icon(color="red")).add_to(m)
m
vehicles = [
    ors.optimization.Vehicle(id=0, profile='driving-car', start=vehicle_start, end=vehicle_start, time_window=[0, 3*60*60]),
    ors.optimization.Vehicle(id=1, profile='driving-car', start=vehicle_start, end=vehicle_start, time_window=[0, 3*60*60])
]
jobs = [ors.optimization.Job(id=index, **job) for index, job in enumerate(coords)]
optimized = client.optimization(jobs=jobs, vehicles=vehicles, geometry=True)
line_colors = ['green', 'orange', 'blue', 'yellow']
for route in optimized['routes']:
    folium.PolyLine(locations=[list(reversed(coords)) for coords in ors.convert.decode_polyline(route['geometry'])['coordinates']], color=line_colors[route['vehicle']]).add_to(m)
    for step in route['steps']:
        if not step['type'] == 'job':
            continue
        folium.Marker(location=list(reversed(step['location'])), popup=f"Arrival time: {math.floor(step['arrival'] / (60*60))} hours {math.floor((step['arrival'] % (60*60)) / 60)} minutes", icon=folium.Icon(color=line_colors[route['vehicle']])).add_to(m)
m